<a href="https://colab.research.google.com/github/kenanmorani/Prediction_Of_Total_Produced_Biogas_Flow/blob/main/Recursive_SVM_Standard_scaled_Feature_selective_for_Biogas_Flow_Prediction_results2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
import warnings
from sklearn import preprocessing

In [2]:
# Mounting my google drive
from google.colab import drive

In [3]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
ss = pd.read_csv('/content/gdrive/MyDrive/Biogaz_Flow/Real_Plant_Data.csv')

In [5]:
del ss['Date'] # Deleting the 'Date' column
ss.head()

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid,Total_Biogaz_Production
0,240,46.26,33.15,787.0,244.36,2113
1,120,44.77,22.02,983.0,413.54,1754
2,200,44.45,22.61,740.0,378.60,1884
3,160,44.31,26.64,761.0,353.66,1768
4,200,50.58,23.23,748.0,343.96,1715


In [6]:
d = preprocessing.minmax_scale(ss) # Applying normalization on the data

In [7]:
d

array([[0.07906977, 0.35562731, 0.49103836, 0.25709628, 0.13314154,
        0.07240799],
       [0.02325581, 0.34417282, 0.3261739 , 0.32112534, 0.22532065,
        0.05513592],
       [0.06046512, 0.34171279, 0.33491335, 0.24174237, 0.2062833 ,
        0.06139043],
       ...,
       [0.34418605, 0.55004613, 0.50866538, 0.14047192, 0.03456035,
        0.25316334],
       [0.47906977, 0.76737392, 0.70967264, 0.13393834, 0.02668715,
        0.22126534],
       [0.48372093, 0.66128536, 0.61161309, 0.35552463, 0.20717142,
        0.33403897]])

In [8]:
names = ss.columns
df = pd.DataFrame(d, columns=names)

In [9]:
df

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid,Total_Biogaz_Production
0,0.079070,0.355627,0.491038,0.257096,0.133142,0.072408
1,0.023256,0.344173,0.326174,0.321125,0.225321,0.055136
2,0.060465,0.341713,0.334913,0.241742,0.206283,0.061390
3,0.041860,0.340637,0.394608,0.248603,0.192695,0.055809
4,0.060465,0.388838,0.344097,0.244356,0.187409,0.053260
...,...,...,...,...,...,...
389,0.390698,0.360394,0.333284,0.100725,0.088485,0.265769
390,0.404651,0.293589,0.271515,0.090546,0.059526,0.238345
391,0.344186,0.550046,0.508665,0.140472,0.034560,0.253163
392,0.479070,0.767374,0.709673,0.133938,0.026687,0.221265


In [10]:
# Identifying the target feature by splitting the dataset
samples = df.filter(['TMF', 'TKM_percent', 'TUKM_percent', 'Alcantine', 'Fatty_Accid'])
scores = df.filter(['Total_Biogaz_Production'])

In [11]:
samples

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid
0,0.079070,0.355627,0.491038,0.257096,0.133142
1,0.023256,0.344173,0.326174,0.321125,0.225321
2,0.060465,0.341713,0.334913,0.241742,0.206283
3,0.041860,0.340637,0.394608,0.248603,0.192695
4,0.060465,0.388838,0.344097,0.244356,0.187409
...,...,...,...,...,...
389,0.390698,0.360394,0.333284,0.100725,0.088485
390,0.404651,0.293589,0.271515,0.090546,0.059526
391,0.344186,0.550046,0.508665,0.140472,0.034560
392,0.479070,0.767374,0.709673,0.133938,0.026687


In [12]:
scores

,Total_Biogaz_Production
0,0.072408
1,0.055136
2,0.061390
3,0.055809
4,0.053260
...,...
389,0.265769
390,0.238345
391,0.253163
392,0.221265


# PBIAS

In [13]:
!pip install spotpy

     |████████████████████████████████| 194kB 8.1MB/s 


In [14]:
import spotpy
import numpy as np
def pbias(model, x_test, y_test):
    x1 = x_test
    x2=y_test
    prediction = model.predict(x1)
    if len(y_test) == len(prediction):
        sim = np.array(prediction)
        obs = np.array(x2)
        print('Pbias is', 100 * (float(np.nansum(sim - obs)) / float(np.nansum(obs))))

    else:
        logging.warning("evaluation and simulation lists does not have the same length.")
        return np.nan

In [15]:
# Identifying the target feature by splitting the dataset
samples = df.filter(['TMF', 'TKM_percent', 'TUKM_percent', 'Alcantine', 'Fatty_Accid'])
scores = df.filter(['Total_Biogaz_Production'])

#Setting a parameter for SVM model
C = 0.1

# Defining the number of features to investigate
nFeatures = len(df.columns) - 1

rfeIndex = nFeatures

In [16]:
#Recursively eliminate features based on the lowest weight
while True:
	#Split into training and testing
	x_train, x_test, y_train, y_test = train_test_split(samples, scores, test_size = 0.50, train_size=0.5)
	
	#Create SVM model using a linear kernel
	model = svm.SVR(kernel='linear', C=C).fit(x_train, y_train)
	coef = model.coef_

	#Print co-efficients of features
	for i in range(0, nFeatures):
		print(samples.columns[i-1],":", coef[0][i-1])
	
	#Find the minimum weight among features and eliminate the feature with the smallest weight
	min = coef[0][0]
	index = 0
	for i in range(0, rfeIndex):
		if min > coef[0][i-1]:
			index = index + 1
			min = coef[0][i-1]
	if len(samples.columns) == 1:
		print("After recursive elimination we have the", samples.columns[index], "feature with a score of:", min)
		pbias(model, x_test, y_test)
		break
	else:
		print ("Lowest feature weight is for", samples.columns[index], "with a value of:", min)
		print ("Dropping feature", samples.columns[index])  

		#Drop the feature in the 'samples' dataframe based on the lowest feature index
		samples.drop(samples.columns[index], axis = 1, inplace = True)
		pbias(model, x_test, y_test)
		print ("\n")
		rfeIndex = rfeIndex - 1
		nFeatures = nFeatures - 1

Fatty_Accid : 0.07760084215455251
TMF : 0.40729979072278655
TKM_percent : 0.011354644598672609
TUKM_percent : 0.06175320475251563
Alcantine : -0.0925062754866956
Lowest feature weight is for Alcantine with a value of: -0.0925062754866956
Dropping feature Alcantine
Pbias is -1198.7747040261872


Fatty_Accid : 0.07691020015884906
TMF : 0.4821209979467472
TKM_percent : 0.04138533239368698
TUKM_percent : 0.10563138955585129
Lowest feature weight is for TUKM_percent with a value of: 0.04138533239368698
Dropping feature TUKM_percent
Pbias is 894.0435084846506


Fatty_Accid : 0.06485428855210126
TMF : 0.4530448209636958
TKM_percent : 0.07431161990866277
Lowest feature weight is for TKM_percent with a value of: 0.06485428855210126
Dropping feature TKM_percent
Pbias is -1908.2242438035835


Fatty_Accid : 0.08658515740525974
TMF : 0.4540989719976616
Lowest feature weight is for Fatty_Accid with a value of: 0.08658515740525974
Dropping feature Fatty_Accid
Pbias is -1773.3035386055924


TMF : 0.43

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

# Average Bias Deviation

In [17]:
# Mean Bias Deviation
def mbd(model, x_test, y_test):
    x1 = x_test
    x2=y_test
    prediction = model.predict(x1)
    predict = np.array(prediction)
    actual = np.array(x2)

    difference = predict - actual
    numerator = np.sum(difference) / len(predict)
    denumerator = np.sum(actual) / len(predict)
    score = float(numerator) / denumerator * 100

    print('Mean Bias Deviation:', score)

In [18]:
# Identifying the target feature by splitting the dataset
samples = df.filter(['TMF', 'TKM_percent', 'TUKM_percent', 'Alcantine', 'Fatty_Accid'])
scores = df.filter(['Total_Biogaz_Production'])

#Setting a parameter for SVM model
C = 0.1

# Defining the number of features to investigate
nFeatures = len(df.columns) - 1

rfeIndex = nFeatures

In [19]:
#Recursively eliminate features based on the lowest weight
while True:
	#Split into training and testing
	x_train, x_test, y_train, y_test = train_test_split(samples, scores, test_size = 0.50, train_size=0.5)
	
	#Create SVM model using a linear kernel
	model = svm.SVR(kernel='linear', C=C).fit(x_train, y_train)
	coef = model.coef_

	#Print co-efficients of features
	for i in range(0, nFeatures):
		print(samples.columns[i-1],":", coef[0][i-1])
	
	#Find the minimum weight among features and eliminate the feature with the smallest weight
	min = coef[0][0]
	index = 0
	for i in range(0, rfeIndex):
		if min > coef[0][i-1]:
			index = index + 1
			min = coef[0][i-1]
	if len(samples.columns) == 1:
		print("After recursive elimination we have the", samples.columns[index], "feature with a score of:", min)
		mbd(model, x_test, y_test)
		break
	else:
		print ("Lowest feature weight is for", samples.columns[index], "with a value of:", min)
		print ("Dropping feature", samples.columns[index])  

		#Drop the feature in the 'samples' dataframe based on the lowest feature index
		samples.drop(samples.columns[index], axis = 1, inplace = True)
		mbd(model, x_test, y_test)
		print ("\n")
		rfeIndex = rfeIndex - 1
		nFeatures = nFeatures - 1

Fatty_Accid : 0.06453099424067361
TMF : 0.4750488291224053
TKM_percent : 0.0841520687984319
TUKM_percent : 0.08691432600319268
Alcantine : -0.016639177246542496
Lowest feature weight is for TUKM_percent with a value of: -0.016639177246542496
Dropping feature TUKM_percent
Mean Bias Deviation: 29.415114512961427


Fatty_Accid : 0.09680450063794449
TMF : 0.5030559382778742
TKM_percent : 0.0938361687746333
Alcantine : -0.008294191133553994
Lowest feature weight is for Fatty_Accid with a value of: -0.008294191133553994
Dropping feature Fatty_Accid
Mean Bias Deviation: -666.5388222745606


Alcantine : -0.02001470868789544
TMF : 0.4476848456455211
TKM_percent : 0.14898877882198586
Lowest feature weight is for TKM_percent with a value of: -0.02001470868789544
Dropping feature TKM_percent
Mean Bias Deviation: -1326.4933572067941


Alcantine : 0.028610108933291484
TMF : 0.48985535133114
Lowest feature weight is for Alcantine with a value of: 0.028610108933291484
Dropping feature Alcantine
Mean B

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

# Index of Agreement

In [20]:
def index_agreement(model, x_test, y_test):
    prediction = model.predict(x_test)
    s = np.ravel(prediction)
    o = np.ravel(y_test)
    ia = 1 -(np.sum((o-s)**2))/(np.sum(
    			(np.abs(s-np.mean(o))+np.abs(o-np.mean(o)))**2))
    print('Index of Agreement:', ia)

In [21]:
# Identifying the target featue by splitting the dataset
samples = df.filter(['TMF', 'TKM_percent', 'TUKM_percent', 'Alcantine', 'Fatty_Accid'])
scores = df.filter(['Total_Biogaz_Production'])

#Setting a parameter for SVM model
C = 0.1

# Defining the number of features to investigate
nFeatures = len(df.columns) - 1

rfeIndex = nFeatures

In [22]:
#Recursively eliminate features based on the lowest weight
while True:
	#Split into training and testing
	x_train, x_test, y_train, y_test = train_test_split(samples, scores, test_size = 0.50, train_size=0.5)
	
	#Create SVM model using a linear kernel
	model = svm.SVR(kernel='linear', C=C).fit(x_train, y_train)
	coef = model.coef_

	#Print co-efficients of features
	for i in range(0, nFeatures):
		print(samples.columns[i-1],":", coef[0][i-1])
	
	#Find the minimum weight among features and eliminate the feature with the smallest weight
	min = coef[0][0]
	index = 0
	for i in range(0, rfeIndex):
		if min > coef[0][i-1]:
			index = index + 1
			min = coef[0][i-1]
	if len(samples.columns) == 1:
		print("After recursive elimination we have the", samples.columns[index], "feature with a score of:", min)
		index_agreement(model, x_test, y_test)
		break
	else:
		print ("Lowest feature weight is for", samples.columns[index], "with a value of:", min)
		print ("Dropping feature", samples.columns[index])  

		#Drop the feature in the 'samples' dataframe based on the lowest feature index
		samples.drop(samples.columns[index], axis = 1, inplace = True)
		index_agreement(model, x_test, y_test)
		print ("\n")
		rfeIndex = rfeIndex - 1
		nFeatures = nFeatures - 1

Fatty_Accid : 0.08328777464876312
TMF : 0.4734699508922544
TKM_percent : 0.021091798257304548
TUKM_percent : 0.03769798569093395
Alcantine : -0.004584005577879309
Lowest feature weight is for Alcantine with a value of: -0.004584005577879309
Dropping feature Alcantine
Index of Agreement: 0.7234718601893062


Fatty_Accid : 0.03456649223286211
TMF : 0.46986307910503217
TKM_percent : 0.056838991429921396
TUKM_percent : 0.07632986158202311
Lowest feature weight is for TKM_percent with a value of: 0.03456649223286211
Dropping feature TKM_percent
Index of Agreement: 0.7571375238414756


Fatty_Accid : 0.04166117159647305
TMF : 0.4389662568829413
TUKM_percent : 0.1290765098032494
Lowest feature weight is for TUKM_percent with a value of: 0.04166117159647305
Dropping feature TUKM_percent
Index of Agreement: 0.7235805943075571


Fatty_Accid : 0.11595697353216539
TMF : 0.4418232211896951
Lowest feature weight is for Fatty_Accid with a value of: 0.11595697353216539
Dropping feature Fatty_Accid
Inde

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin